In [1]:
import time
import json
import os
import numpy as np
import flwr as fl
import pickle
import multiprocessing

from math import floor

from hydra import initialize, compose
from omegaconf import OmegaConf, DictConfig

from logging import INFO, DEBUG
from flwr.common.logger import log

from keras import layers, models, Input, regularizers, optimizers
from keras.callbacks import TensorBoard, EarlyStopping

from src.models.evaluation_metrics import custom_acc_mc, custom_acc_binary
from src.data.dataset_info import datasets
from src.read_clients import read_clients

with initialize(version_base=None, config_path="conf/"):
    cfg = compose(config_name='config.yaml')
    print(OmegaConf.to_yaml(cfg))


dataset = datasets[0]

folder_path = "./datasets/gdlc/"
# folder_path = "./datasets/dbp/"

lr_decay = True
early_stopping = False

pca = True
digraph_centralities = False
multi_graph_centralities = False

learning_rate = 0.001
LAMBD_1 = 0.0001
LAMBD_2 = 0.001

dtime = time.strftime("%Y%m%d-%H%M%S")
dtime

multi_class: false
with_network_features: false
n_clients: 5
n_rounds: 20
config_fit:
  momentum: 0.9
  local_epochs: 3
  batch_size: 256



'20240905-122059'

In [2]:
clients_paths = [
    folder_path + "client_0.parquet",
    folder_path + "client_1.parquet",
    folder_path + "client_2.parquet",
    folder_path + "client_3.parquet",
    folder_path + "client_4.parquet",
    folder_path + "client_5.parquet",
    folder_path + "client_6.parquet",
    folder_path + "client_7.parquet"
    # folder_path + "test.parquet"
]

# Data Loading and Preprocessing

In [3]:
with open(folder_path + "added_columns.pkl", 'rb') as f:
    centralities_columns, pca_columns = pickle.load(f)
centralities_columns, pca_columns

([['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_degree',
   'dst_degree',
   'src_local_degree',
   'dst_local_degree',
   'src_eigenvector',
   'dst_eigenvector',
   'src_closeness',
   'dst_closeness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',
   'src_Comm',
   'dst_Comm'],
  ['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',
   'src_Comm',
   'dst_Comm'],
  ['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',


In [4]:
# the input dimension of the training set
# input_dim = df.shape[1] - len(drop_columns) - len(weak_columns) - 1  # for the label_column
  
# specifying the number of classes, since it is different from one dataset to another and also if binary or multi-class classification
classes_set = {"benign", "attack"}
labels_names = {0: "benign", 1: "attack"}
num_classes = 2
if cfg.multi_class:
    with open(folder_path + "labels_names.pkl", 'rb') as f:
        labels_names, classes_set = pickle.load(f)
    num_classes = len(classes_set)
    
labels_names = {int(k): v for k, v in labels_names.items()}

print(f"==>> classes_set: {classes_set}")
print(f"==>> num_classes: {num_classes}")
print(f"==>> labels_names: {labels_names}")

==>> classes_set: {'attack', 'benign'}
==>> num_classes: 2
==>> labels_names: {0: 'benign', 1: 'attack'}


# Model

In [5]:
def create_keras_model(input_shape, alpha = learning_rate):
    model = models.Sequential()
    
    model.add(layers.Conv1D(80, kernel_size=3,
                activation="relu", input_shape=(input_shape, 1), kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    # .L1L2(l1=LAMBD_1, l2=LAMBD_2)
    model.add(layers.Conv1D(80, 3, activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    
    # model.add(layers.LSTM(units=80,
    #                         activation='relu',
    #                         kernel_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         recurrent_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         bias_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         return_sequences=False,
    #                         ))
    # model.add(layers.LayerNormalization(axis=1))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(80,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))

    if cfg.multi_class:
        model.add(layers.Dense(num_classes, activation='softmax'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    else:
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    
    
    return model


In [6]:
model = create_keras_model(80)
model.summary()

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 78, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 39, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 39, 80)         │            78 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 37, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 18, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 18, 80)         │            36 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1440)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       288,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 365,235 (1.39 MB)

 Trainable params: 365,235 (1.39 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
results_final = {}

# results_final["model"] = model.to_json()
results_final["model"] = {}
results_final["configuration"] = {
    "folder_path": folder_path,
    "lr_decay": lr_decay,
    "early_stopping": early_stopping,
    "pca": pca,
    "digraph_centralities": digraph_centralities,
    "multi_graph_centralities": multi_graph_centralities,
    "learning_rate": learning_rate,
    "LAMBD_1": LAMBD_1,
    "LAMBD_2": LAMBD_2,
    "cfg": OmegaConf.to_container(cfg)
}

results_final["baseline"] = {}
results_final["baseline"]["accuracy"] = {}
results_final["baseline"]["f1s"] = {}

results_final["PCA"] = {}
results_final["PCA"]["accuracy"] = {}
results_final["PCA"]["f1s"] = {}

results_final["digraph"] = {}
results_final["digraph"]["accuracy"] = {}
results_final["digraph"]["f1s"] = {}

results_final["multidigraph"] = {}
results_final["multidigraph"]["accuracy"] = {}
results_final["multidigraph"]["f1s"] = {}

results_final

{'model': {},
 'configuration': {'folder_path': './datasets/gdlc/',
  'lr_decay': True,
  'early_stopping': False,
  'pca': True,
  'digraph_centralities': False,
  'multi_graph_centralities': False,
  'learning_rate': 0.001,
  'LAMBD_1': 0.0001,
  'LAMBD_2': 0.001,
  'cfg': {'multi_class': False,
   'with_network_features': False,
   'n_clients': 5,
   'n_rounds': 20,
   'config_fit': {'momentum': 0.9, 'local_epochs': 3, 'batch_size': 256}}},
 'baseline': {'accuracy': {}, 'f1s': {}},
 'PCA': {'accuracy': {}, 'f1s': {}},
 'digraph': {'accuracy': {}, 'f1s': {}},
 'multidigraph': {'accuracy': {}, 'f1s': {}}}

# FL Settings

In [8]:
class FLClient(fl.client.NumPyClient):
    def __init__(self, logdir, x_train, y_train, x_val, y_val, x_test, y_test, input_dim):
        self.logdir = logdir
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val  
        self.x_test, self.y_test = x_test, y_test
        self.input_dim = input_dim
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        self.model = create_keras_model(input_shape=self.input_dim, alpha=lr)
        self.set_parameters(parameters, config)

        tensorboard_callback = TensorBoard(log_dir=self.logdir)
        callbacks=[tensorboard_callback]
        if early_stopping:
            early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            callbacks.append(early_stopping_callback)

        history = self.model.fit(self.x_train, self.y_train,
                                epochs=config["local_epochs"],
                                batch_size=config["batch_size"],
                                validation_data=(self.x_val, self.y_val),  
                                verbose=0,
                                callbacks=callbacks)

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [9]:
def generate_client_fn(data, simulation_name, input_dim):
    def client_fn(cid: str):
        i = int(cid)
        logdir = "logs/scalars/{}/{}/client_{}".format(dtime, simulation_name, cid)
        return FLClient(
            logdir,
            data[i][0],  # x_train
            data[i][1],  # y_train
            data[i][2],  # x_val
            data[i][3],  # y_val
            data[i][4],  # x_test
            data[i][5],   # y_test
            input_dim
        ).to_client()

    return client_fn

In [10]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if lr_decay and server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server, input_dim, simulation_name, results, test_by_class):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/{}/server".format(dtime, simulation_name) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final[simulation_name]["accuracy"][server_round] = scores["accuracy"]
        results_final[simulation_name]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [11]:
def weighted_average(metrics):
    # print(f"==>> weighted_average: {metrics}")

    return metrics
    # total_examples = 0
    # federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    # for num_examples, m in metrics:
    #     for k, v in m.items():
    #         federated_metrics[k] += num_examples * v
    #     total_examples += num_examples
    # return {k: v / total_examples for k, v in federated_metrics.items()}

# BaseLine

In [12]:
simulation_name = "baseline"

client_data, test, test_labels, test_by_class, input_dim = read_clients(
    folder_path, clients_paths, dataset.label_col, dataset.class_col, dataset.class_num_col, centralities_columns, pca_columns, dataset.drop_columns, dataset.weak_columns, cfg.multi_class)

In [13]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - baseline"
results["dtime"] = dtime
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - baseline',
 'dtime': '20240905-122059',
 'multi_class': False,
 'learning_rate': 0.001,
 'dataset_name': 'cic_ton_iot',
 'num_classes': 2,
 'labels_names': {0: 'benign', 1: 'attack'},
 'input_dim': 38,
 'scores': {'server': {},
  'clients': {},
  'accuracy': {},
  'f1s': {},
  'test_by_class': {'accuracy': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': {},
    'DoS GoldenEye': {},
    'PortScan': {},
    'DoS slowloris': {},
    'FTP-Patator': {},
    'SSH-Patator': {},
    'Bot': {},
    'DoS Slowhttptest': {},
    'bruteforce': {},
    'Infiltration': {},
    'Web Attack � Sql Injection': {},
    'Heartbleed': {}},
   'f1s': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hu

In [14]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    # fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels, input_dim, simulation_name, results, test_by_class),
)  # a function to run on the server side to evaluate the global model.

In [15]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(client_data, simulation_name, input_dim),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_cpus": 1,
        "num_gpus": 0.0,
    },
)

INFO flwr 2024-09-05 12:21:13,928 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-09-05 12:21:18,963	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-09-05 12:21:21,755 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 15810870067.0, 'memory': 31621740135.0, 'node:__internal_head__': 1.0, 'CPU': 32.0}
INFO flwr 2024-09-05 12:21:21,756 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-09-05 12:21:21,758 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-09-05 12:21:21,784 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-09-05 12:21:21,786 | server.py:89 | Initializing global parameters
INFO flwr 2024-09-05 12:21:21,787 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.5334 - loss: 1.7414
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.7141 - loss: 0.9031
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

...

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.9043 - loss: 0.2752
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.8986 - loss: 0.3076
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

In [16]:
print(f"==>> history: {history}")
print(f"==>> end of history")

==>> history: History (loss, centralized):
	round 0: 1.7265715599060059
	round 1: 0.848837673664093
	round 2: 0.6468979716300964
	round 3: 0.5039086937904358
	round 4: 0.6309142112731934
	round 5: 0.7982929944992065
	round 6: 0.5059230923652649
	round 7: 0.42130908370018005
	round 8: 0.4732332229614258
	round 9: 0.40588560700416565
	round 10: 0.4282730221748352
	round 11: 0.4247918725013733
	round 12: 0.42171207070350647
	round 13: 0.4633745849132538
	round 14: 0.48121821880340576
	round 15: 0.39264124631881714
	round 16: 0.39520543813705444
	round 17: 0.339618444442749
	round 18: 0.3282938301563263
	round 19: 0.33917000889778137
	round 20: 0.35006284713745117
History (metrics, centralized):
{'accuracy': [(0, 0.503389835357666), (1, 0.7199515700340271), (2, 0.7232339978218079), (3, 0.7685281038284302), (4, 0.7260739207267761), (5, 0.7232063412666321), (6, 0.8139286041259766), (7, 0.8336144685745239), (8, 0.7976144552230835), (9, 0.8366267085075378), (10, 0.8204721212387085), (11, 0.854

In [17]:
# creating the directories if they don't exist
if not os.path.isdir('./results'):
    os.mkdir('./results')

# creating the directories if they don't exist
if not os.path.isdir('./results/{}'.format(dtime)):
    os.mkdir('./results/{}'.format(dtime))

# if not os.path.isdir('./results/{}'.format(dataset_name)):
#     os.mkdir('./results/{}'.format(dataset_name))

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

filename = ('./results/{}/baseline.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [18]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# FL - PCA

In [19]:
if pca:
    simulation_name = "PCA"
    client_data, test, test_labels, test_by_class, input_dim = read_clients(
        folder_path, clients_paths, dataset.label_col, dataset.class_col, dataset.class_num_col, centralities_columns, None, dataset.drop_columns, dataset.weak_columns, cfg.multi_class)

In [20]:
if pca:
    results = {}  # a dictionary that will contain all the options and results of models
    # add all options to the results dictionary, to know what options selected for obtained results
    results["configuration"] = "2dt - PCA"
    results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
    results["multi_class"] = cfg.multi_class
    results["learning_rate"] = learning_rate
    results["dataset_name"] = dataset.name
    results["num_classes"] = num_classes
    results["labels_names"] = labels_names
    results["input_dim"] = input_dim

    results["scores"] = {}
    results["scores"]["server"] = {}
    results["scores"]["clients"] = {}
    results["scores"]["accuracy"] = {}
    results["scores"]["f1s"] = {}

    if not cfg.multi_class:
        results["scores"]["test_by_class"] = {}
        results["scores"]["test_by_class"]["accuracy"] = {}
        results["scores"]["test_by_class"]["f1s"] = {}
        for k in test_by_class.keys():
            results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
            results["scores"]["test_by_class"]["accuracy"][k] = {}   
            results["scores"]["test_by_class"]["f1s"][k] = {}    
            
    results

In [21]:
if pca:
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
        min_fit_clients=len(client_data),  # number of clients to sample for fit()
        fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
        min_evaluate_clients=0,  # number of clients to sample for evaluate()
        min_available_clients=len(client_data),  # total clients in the simulation
        # fit_metrics_aggregation_fn = weighted_average,
        # evaluate_metrics_aggregation_fn = weighted_average,
        on_fit_config_fn=get_on_fit_config(
            cfg.config_fit
        ),  # a function to execute to obtain the configuration to send to the clients during fit()
        evaluate_fn=get_evaluate_fn(test, test_labels, input_dim, simulation_name, results, test_by_class),
    )  # a function to run on the server side to evaluate the global model.


In [22]:
if pca:
    import multiprocessing
    from math import floor
    history = fl.simulation.start_simulation(
        client_fn=generate_client_fn(client_data, simulation_name, input_dim),  # a function that spawns a particular client
        # num_clients=cfg.n_clients,  # total number of clients
        num_clients=len(client_data),  # total number of clients
        config=fl.server.ServerConfig(
            num_rounds=cfg.n_rounds
            # num_rounds=5
        ),  # minimal config for the server loop telling the number of rounds in FL
        strategy=strategy,  # our strategy of choice
        client_resources={
            # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
            "num_cpus": 1,
            "num_gpus": 0.0,
        },
    )

INFO flwr 2024-09-05 14:02:28,016 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)


2024-09-05 14:02:38,332	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-09-05 14:02:40,855 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 15550521753.0, 'memory': 31101043508.0, 'node:__internal_head__': 1.0, 'CPU': 32.0}
INFO flwr 2024-09-05 14:02:40,858 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-09-05 14:02:40,859 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-09-05 14:02:40,888 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-09-05 14:02:40,890 | server.py:89 | Initializing global parameters
INFO flwr 2024-09-05 14:02:40,891 | server.py:276 | Requesting initial parameters from one random client
(pid=11932) 2024-09-05 14:02:44.699376: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.3866 - loss: 1.8299
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.5974 - loss: 1.4977
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step


...

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 25s 14ms/step - accuracy: 0.9989 - loss: 0.0039
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 0.9921 - loss: 0.0918
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

In [23]:
if pca:
    print(f"==>> history: {history}")
    print(f"==>> end of history")

==>> history: History (loss, centralized):
	round 0: 1.7186343669891357
	round 1: 1.222705364227295
	round 2: 0.6811822056770325
	round 3: 1.2611004114151
	round 4: 1.1359401941299438
	round 5: 1.1062532663345337
	round 6: 1.308359980583191
	round 7: 0.3317229151725769
	round 8: 0.30123385787010193
	round 9: 0.2464710921049118
	round 10: 0.26225170493125916
	round 11: 0.46221718192100525
	round 12: 1.2502905130386353
	round 13: 0.13938871026039124
	round 14: 0.8019105195999146
	round 15: 0.03566907346248627
	round 16: 0.27769196033477783
	round 17: 0.37326496839523315
	round 18: 0.3043750822544098
	round 19: 0.01259738951921463
	round 20: 0.44467291235923767
History (metrics, centralized):
{'accuracy': [(0, 0.44517457485198975), (1, 0.6499447822570801), (2, 0.5416277647018433), (3, 0.681307315826416), (4, 0.6644784212112427), (5, 0.4915812611579895), (6, 0.6895420551300049), (7, 0.9473369717597961), (8, 0.9626257419586182), (9, 0.9615727663040161), (10, 0.9524871110916138), (11, 0.9006

In [24]:
if pca:
    filename = ('./results/{}/pca.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results, cls=NumpyEncoder))
    outfile.close()

In [25]:
if pca:
    filename = ('./results/{}/results_final.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
    outfile.close()

# Centralities - DiGraph

In [26]:
if digraph_centralities:
    simulation_name = "digraph"
    client_data, test, test_labels, test_by_class, input_dim = read_clients(
        folder_path, clients_paths, dataset.label_col, dataset.class_col, dataset.class_num_col, None, pca_columns, dataset.drop_columns, dataset.weak_columns, cfg.multi_class)

In [27]:
if digraph_centralities:
    results = {}  # a dictionary that will contain all the options and results of models
    # add all options to the results dictionary, to know what options selected for obtained results
    results["configuration"] = "2dt - Centralities - DiGraph"
    results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
    results["multi_class"] = cfg.multi_class
    results["learning_rate"] = learning_rate
    results["dataset_name"] = dataset.name
    results["num_classes"] = num_classes
    results["labels_names"] = labels_names
    results["input_dim"] = input_dim

    results["scores"] = {}
    results["scores"]["server"] = {}
    results["scores"]["clients"] = {}
    results["scores"]["accuracy"] = {}
    results["scores"]["f1s"] = {}

    if not cfg.multi_class:
        results["scores"]["test_by_class"] = {}
        results["scores"]["test_by_class"]["accuracy"] = {}
        results["scores"]["test_by_class"]["f1s"] = {}
        for k in test_by_class.keys():
            results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
            results["scores"]["test_by_class"]["accuracy"][k] = {}   
            results["scores"]["test_by_class"]["f1s"][k] = {}    
            
    results

In [28]:
if digraph_centralities:
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
        min_fit_clients=len(client_data),  # number of clients to sample for fit()
        fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
        min_evaluate_clients=0,  # number of clients to sample for evaluate()
        min_available_clients=len(client_data),  # total clients in the simulation
        # fit_metrics_aggregation_fn = weighted_average,
        # evaluate_metrics_aggregation_fn = weighted_average,
        on_fit_config_fn=get_on_fit_config(
            cfg.config_fit
        ),  # a function to execute to obtain the configuration to send to the clients during fit()
        evaluate_fn=get_evaluate_fn(test, test_labels, input_dim, simulation_name, results, test_by_class),
    )  # a function to run on the server side to evaluate the global model.


In [29]:
if digraph_centralities:
    import multiprocessing
    from math import floor
    history = fl.simulation.start_simulation(
        client_fn=generate_client_fn(client_data, simulation_name, input_dim),  # a function that spawns a particular client
        # num_clients=cfg.n_clients,  # total number of clients
        num_clients=len(client_data),  # total number of clients
        config=fl.server.ServerConfig(
            num_rounds=cfg.n_rounds
            # num_rounds=5
        ),  # minimal config for the server loop telling the number of rounds in FL
        strategy=strategy,  # our strategy of choice
        client_resources={
            # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
            "num_cpus": 1,
            "num_gpus": 0.0,
        },
    )

In [30]:
if digraph_centralities:
    print(f"==>> history: {history}")
    print(f"==>> end of history")

In [31]:
if digraph_centralities:
    filename = ('./results/{}/digraph.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results, cls=NumpyEncoder))
    outfile.close()

In [32]:
if digraph_centralities:
    filename = ('./results/{}/results_final.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
    outfile.close()

# Centralities - MultiDiGraph

In [33]:
if multi_graph_centralities:
    simulation_name = "multidigraph"
    client_data, test, test_labels, test_by_class, input_dim = read_clients(
        folder_path, clients_paths, dataset.label_col, dataset.class_col, dataset.class_num_col, centralities_columns, pca_columns, dataset.drop_columns, dataset.weak_columns, cfg.multi_class)

In [34]:
if multi_graph_centralities:
    results = {}  # a dictionary that will contain all the options and results of models
    # add all options to the results dictionary, to know what options selected for obtained results
    results["configuration"] = "2dt - Centralities - MultiDiGraph"
    results["dtime"] = dtime
    results["multi_class"] = cfg.multi_class
    results["learning_rate"] = learning_rate
    results["dataset_name"] = dataset.name
    results["num_classes"] = num_classes
    results["labels_names"] = labels_names
    results["input_dim"] = input_dim

    results["scores"] = {}
    results["scores"]["server"] = {}
    results["scores"]["clients"] = {}
    results["scores"]["accuracy"] = {}
    results["scores"]["f1s"] = {}

    if not cfg.multi_class:
        results["scores"]["test_by_class"] = {}
        results["scores"]["test_by_class"]["accuracy"] = {}
        results["scores"]["test_by_class"]["f1s"] = {}
        for k in test_by_class.keys():
            results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
            results["scores"]["test_by_class"]["accuracy"][k] = {}   
            results["scores"]["test_by_class"]["f1s"][k] = {}    
            
    results

In [35]:
if multi_graph_centralities:
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
        min_fit_clients=len(client_data),  # number of clients to sample for fit()
        fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
        min_evaluate_clients=0,  # number of clients to sample for evaluate()
        min_available_clients=len(client_data),  # total clients in the simulation
        # fit_metrics_aggregation_fn = weighted_average,
        # evaluate_metrics_aggregation_fn = weighted_average,
        on_fit_config_fn=get_on_fit_config(
            cfg.config_fit
        ),  # a function to execute to obtain the configuration to send to the clients during fit()
        evaluate_fn=get_evaluate_fn(test, test_labels, input_dim, simulation_name, results, test_by_class),
    )  # a function to run on the server side to evaluate the global model.

In [36]:
if multi_graph_centralities:
    history = fl.simulation.start_simulation(
        client_fn=generate_client_fn(client_data, simulation_name, input_dim),  # a function that spawns a particular client
        # num_clients=cfg.n_clients,  # total number of clients
        num_clients=len(client_data),  # total number of clients
        config=fl.server.ServerConfig(
            num_rounds=cfg.n_rounds
            # num_rounds=5
        ),  # minimal config for the server loop telling the number of rounds in FL
        strategy=strategy,  # our strategy of choice
        client_resources={
            # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
            "num_cpus": 1,
            "num_gpus": 0.0,
        },
    )

In [37]:
if multi_graph_centralities:
    print(f"==>> history: {history}")
    print(f"==>> end of history")

In [38]:
if multi_graph_centralities:
    filename = ('./results/{}/multidigraph.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results, cls=NumpyEncoder))
    outfile.close()

In [39]:
if multi_graph_centralities:
    filename = ('./results/{}/results_final.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
    outfile.close()